In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json

In [2]:
with open('203.html', 'r') as file:
    html_doc = file.read().replace('\n', '')

In [3]:
soup = BeautifulSoup(html_doc, 'html.parser')

In [4]:
primeDocs = soup.find_all(id='primeDocs')

In [5]:
kpi = pd.read_html('203.html')[1:]
primeDocs_pd = pd.DataFrame(primeDocs[0])
primeDocs_pd.columns = ["html_mkb"]

In [6]:
#primeDocs_pd["МКБ"] = primeDocs_pd["text"].astype(str).str.findall("МКБ.*\)").str.get(0).str.replace('\)', '')
#primeDocs_pd["МКБ"] = primeDocs_pd["text"].astype(str).str.findall(". Критерии").str.get(0).str.replace('\)', '')
primeDocs_pd["MKB"] = primeDocs_pd["html_mkb"].astype(str).str.findall("10:.*\)").str.get(0).str.replace('\)', '').str.replace('10:', '')
#primeDocs_pd["num"] = primeDocs_pd["text"].astype(str).str.findall(">3\.\d*\.\d*\.").str.get(0).str.replace('>', '')

In [7]:
primeDocs_pd = primeDocs_pd.dropna().reset_index()
#primeDocs_pd = primeDocs_pd.dropna(subset=['num']).reset_index()

In [8]:
kpi_list = []
for k in kpi:
    #kpi_list.append(k['Критерии качества'].to_list())
    kpi_list.append(k.iloc[:,1].to_list())

In [9]:
primeDocs_pd['kpi'] = kpi_list

In [10]:
primeDocs_pd['MKB'] = primeDocs_pd['MKB'].str.split(';')

In [11]:
#primeDocs_pd.to_excel('203.xls')

In [12]:
primeDocs_pd[['MKB', 'kpi']]

,MKB,kpi
0,"[ А49.8 - А49.9, R50.0 - R50.1]","[Выполнена отоскопия, Выполнен общий (клиничес..."
1,"[ В15.9, В16.1, В16.9, В17.1, В17.2, В17....",[Выполнен осмотр врачом-инфекционистом не позд...
2,"[ В15.0, В16.0, В16.2]",[Выполнен осмотр врачом-инфекционистом не позд...
3,[ А98.5],[Выполнен осмотр врачом-инфекционистом и/или в...
4,"[ А02.0, А02.2+, А02.8, А02.9, А03, А04, ...","[Выполнено измерение массы тела, Выполнена оце..."
...,...,...
226,"[ S43, S46, S53, S56, S63, S66, S73, S7...",[Выполнен осмотр врачом-травматологом-ортопедо...
227,"[ S02.4, S02.6]",[Выполнен осмотр врачом-челюстно-лицевым хирур...
228,"[ Т42, Т43]",[Выполнен осмотр врачом-токсикологом и/или вра...
229,[ Т58],[Выполнен осмотр врачом-токсикологом и/или вра...


In [13]:
mkb = {}
for i in primeDocs_pd.index:
    for k in primeDocs_pd.loc[i,'MKB']:
        mkb[k] = primeDocs_pd.loc[i,'kpi']    

In [14]:
#with open('203.json', 'w', encoding='utf-8') as outfile:
#    json.dump(mkb, outfile, ensure_ascii=False, indent=4)

In [23]:
kpi[0]['Критерии качества'].to_list()

['Выполнена отоскопия',
 'Выполнен общий (клинический) анализ крови развернутый не позднее 24 часов от момента поступления в стационар',
 'Выполнено исследование уровня С-реактивного белка или прокальцитонина в крови',
 'Выполнен общий анализ мочи не позднее 24 часов от момента поступления',
 'Выполнено бактериологическое исследование крови на стерильность с определением чувствительности возбудителя к антибиотикам и другим лекарственным препаратам (при наличии лабораторных маркеров бактериальной инфекции)',
 'Выполнено бактериологическое исследование мочи с определением чувствительности возбудителя к антибиотикам и другим лекарственным препаратам (при выявлении лейкоцитурии и/или бактериурии и/или нитритов в моче)',
 'Выполнена рентгенография органов грудной клетки (при наличии лабораторных маркеров бактериальной инфекции)',
 'Проведена терапия антибактериальными лекарственными препаратами (при подтверждении бактериальной инфекции и/или при наличии лабораторных маркеров бактериальной и

In [38]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
#list_base =leads['reason_clear'].unique()
def frase_detect (list_base, frase):
    #list_match = process.extract(frase, leads['reason_clear'].unique(), scorer=fuzz.token_set_ratio, limit=30)
    list_match = process.extract(frase, list_base, scorer=fuzz.token_set_ratio, limit=30)
    return list_match[0]

#frase = 'напомнить'
#str(frase_detect (list_base, frase))

In [39]:
med_service = pd.read_excel('1.2.643.5.1.13.13.11.1070_2.10.xlsx').drop(0)

In [40]:
list_base = med_service['NAME'].to_list()

In [41]:
frase = 'отоскопия'
str(frase_detect (list_base, frase))

"('Осмотр органа слуха (отоскопия)', 100)"

In [52]:
from tqdm.notebook import trange, tqdm

In [48]:
kpi_list = kpi[0].iloc[:,1].to_list()

In [53]:
service_list = []
for frase in tqdm(kpi_list):
    service_list.append(frase_detect (list_base, frase)[0])

In [54]:
service_list

['Видеоотоскопия',
 'Общий (клинический) анализ крови',
 'Исследование уровня прокальцитонина в крови',
 'Общий (клинический) анализ мочи',
 'Определением чувствительности лейкоцитов крови к препаратам интерферона',
 'Исследование уровня нитритов в моче',
 'Прицельная рентгенография органов грудной клетки',
 'Заушные блокады с лекарственными препаратами',
 'Заушные блокады с лекарственными препаратами']

In [55]:
df = kpi[0].copy()

In [57]:
df['service']=service_list

In [63]:
pd.set_option('display.max_colwidth', 1000)

In [64]:
df[['Критерии качества','service']]

,Критерии качества,service
0,Выполнена отоскопия,Видеоотоскопия
1,Выполнен общий (клинический) анализ крови развернутый не позднее 24 часов от момента поступления в стационар,Общий (клинический) анализ крови
2,Выполнено исследование уровня С-реактивного белка или прокальцитонина в крови,Исследование уровня прокальцитонина в крови
3,Выполнен общий анализ мочи не позднее 24 часов от момента поступления,Общий (клинический) анализ мочи
4,Выполнено бактериологическое исследование крови на стерильность с определением чувствительности возбудителя к антибиотикам и другим лекарственным препаратам (при наличии лабораторных маркеров бактериальной инфекции),Определением чувствительности лейкоцитов крови к препаратам интерферона
5,Выполнено бактериологическое исследование мочи с определением чувствительности возбудителя к антибиотикам и другим лекарственным препаратам (при выявлении лейкоцитурии и/или бактериурии и/или нитритов в моче),Исследование уровня нитритов в моче
6,Выполнена рентгенография органов грудной клетки (при наличии лабораторных маркеров бактериальной инфекции),Прицельная рентгенография органов грудной клетки
7,Проведена терапия антибактериальными лекарственными препаратами (при подтверждении бактериальной инфекции и/или при наличии лабораторных маркеров бактериальной инфекции),Заушные блокады с лекарственными препаратами
8,Выполнена смена терапии антибактериальными лекарственными препаратами не позднее 72 часов от момента начала антибактериальной терапии (при отсутствии снижения температуры тела),Заушные блокады с лекарственными препаратами
